# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [7]:
import os
import gzip
import urllib.request
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader


# Custom dataset class that is used to manually load the MNIST data

class FashionMNISTCustom(Dataset):
    def __init__(self, image_path, label_path):
        with gzip.open(label_path, 'rb') as lbpath:
            lbpath.read(8)
            self.labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

        with gzip.open(image_path, 'rb') as imgpath:
            imgpath.read(16)
            images = np.frombuffer(imgpath.read(), dtype=np.uint8)
            self.images = images.reshape(-1, 28, 28).astype(np.float32) / 255.0

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = torch.tensor(self.images[idx]).unsqueeze(0)
        y = torch.tensor(self.labels[idx]).long()
        return x, y

  # URL source and list of files to download
  ###### Was very lost on this part. Used a combination of ChatGPT and stack overflow in order to access .gz type files.

base_url = "https://github.com/zalandoresearch/fashion-mnist/tree/master/data/fashion"
files = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"
]

# Create 'data' folder if it doesn't exist
os.makedirs("data", exist_ok=True)

# Loop through files and download if not already present
for file in files:
    url = base_url + file
    dest = os.path.join("data", file)
    if not os.path.exists(dest):
        print(f"Downloading {file}...")
        urllib.request.urlretrieve(url, dest)
    else:
        print(f"{file} already exists.")


# Using a convolutional neural networks for image classification

class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

  # Load training data
train_data = FashionMNISTCustom("data/train-images-idx3-ubyte.gz", "data/train-labels-idx1-ubyte.gz")
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

model = FashionCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop - played around with epochs a little. It seemed that anything more than 5 was getting negliable additional accuracy for the time it added.
for epoch in range(5):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")

# Save trained weights to file
torch.save(model.state_dict(), "fashion_model.pt")
model = FashionCNN()
model.load_state_dict(torch.load("fashion_model.pt"))
model.eval()

# Loading test data
test_data = FashionMNISTCustom("data/t10k-images-idx3-ubyte.gz", "data/t10k-labels-idx1-ubyte.gz")
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Track number of correct predictions
correct = 0
total = 0

# Evalute trained model
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print test accuracy
print(f"Test Accuracy: {100 * correct / total:.2f}%")


train-images-idx3-ubyte.gz already exists.
train-labels-idx1-ubyte.gz already exists.
t10k-images-idx3-ubyte.gz already exists.
t10k-labels-idx1-ubyte.gz already exists.
Epoch 1, Loss: 444.1690
Epoch 2, Loss: 284.7474
Epoch 3, Loss: 244.3406
Epoch 4, Loss: 214.4134
Epoch 5, Loss: 192.5802
Test Accuracy: 91.27%
